In [1]:
import pandas as pd
import os
import datetime as dt

In [15]:

# Define the directory path
directory_path = "/Users/robertgarybuckley/Documents/Viz/NPS Project/"

# List all files in the directory
files = os.listdir(directory_path)

file_path = directory_path+files[0]


In [31]:
park_name = pd.read_csv(file_path, nrows=1).iloc[0,0]

In [41]:
#read data
df = pd.read_csv(file_path, skiprows=3)

# Drop the last column from the DataFrame
df = df.drop(df.columns[-1], axis=1)

In [49]:
df_pivot = df.melt(id_vars='Year', var_name='Month', value_name='Value').dropna()


In [61]:
# Function to extract month number from month name
def extract_month_number(month_name):
    return pd.to_datetime(month_name, format='%b').month

# Add a new column "Month_Numbers" by applying the function
df_pivot['Month_Numbers'] = df_pivot['Month'].apply(extract_month_number)

In [62]:
df_pivot

,Year,Month,Value,Month_Numbers
0,2024,JAN,"13,130",1
1,2023,JAN,"17,411",1
2,2022,JAN,"17,539",1
3,2021,JAN,"23,417",1
4,2020,JAN,"13,651",1
...,...,...,...,...
547,1983,DEC,"6,273",12
548,1982,DEC,"6,128",12
549,1981,DEC,"7,911",12
550,1980,DEC,"4,391",12


In [ ]:


# Function to extract month from string
def extract_month_from_string(value):
    if pd.notnull(value):
        return int(value.split(',')[0].replace(',', ''))

# Add a new column "Month_Numbers" by applying the function to each column
df['Month_Numbers'] = df.apply(lambda row: extract_month_from_string(row[row.notnull()].iloc[0]), axis=1)

# Display the modified DataFrame
print(df)
